# Aggregation on CSV Data

Here are the task reated to aggregations.

- Get count by order Status
- Get count by order month and then by order_status. We need to generate order month using order date, using apply on dataframe 

In [1]:
import pandas as pd 

column_list = ["order_id","order_date", "order_customer_id","order_status"]
file_path = 'E:/Projects/Data_Engineering/Data-Engineering/data/retail_db/orders/part-00000'
orders = pd.read_csv(
                    file_path,
                    names= column_list
)
orders

,order_id,order_date,order_customer_id,order_status
0,1,2013-07-25 00:00:00.0,11599,CLOSED
1,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
2,3,2013-07-25 00:00:00.0,12111,COMPLETE
3,4,2013-07-25 00:00:00.0,8827,CLOSED
4,5,2013-07-25 00:00:00.0,11318,COMPLETE
...,...,...,...,...
68878,68879,2014-07-09 00:00:00.0,778,COMPLETE
68879,68880,2014-07-13 00:00:00.0,1117,COMPLETE
68880,68881,2014-07-19 00:00:00.0,2518,PENDING_PAYMENT
68881,68882,2014-07-22 00:00:00.0,10000,ON_HOLD


In [2]:
orders.columns

Index(['order_id', 'order_date', 'order_customer_id', 'order_status'], dtype='object')

In [7]:
# In order to achive the aggregation we need to first group the data
# To group the data we will use function groupby. 
# groupby function will take argumnets as key column name on the base we need to group the data 
# after the groupby we will then use agg function to aggregate the data.
# agg function will take argumnet as aggregation function name such as cout, sum etc.

orders. \
        groupby('order_status'). \
        agg('count')


,order_id,order_date,order_customer_id
order_status,,,
CANCELED,1428,1428,1428
CLOSED,7556,7556,7556
COMPLETE,22899,22899,22899
ON_HOLD,3798,3798,3798
PAYMENT_REVIEW,729,729,729
PENDING,7610,7610,7610
PENDING_PAYMENT,15030,15030,15030
PROCESSING,8275,8275,8275
SUSPECTED_FRAUD,1558,1558,1558


In [10]:
# If we did not specify the key column name for which we need to see the aggregation then it will show the aggreation for all the columns
orders. \
        groupby('order_status')['order_id']. \
        agg(order_count = 'count') # type: ignore

,order_count
order_status,
CANCELED,1428
CLOSED,7556
COMPLETE,22899
ON_HOLD,3798
PAYMENT_REVIEW,729
PENDING,7610
PENDING_PAYMENT,15030
PROCESSING,8275
SUSPECTED_FRAUD,1558


In [11]:
# if we want to aggregate the data on the column which is not in the dataframe then we can add the column using "apply" function.
# apply function will take a function as argument on which logic the column will be generated. 

# add a column order_month as we will make the aggregation based on this and order_status.
# Note:- query will used for filter the data and work on entire dataframe whereas apply will work on row level transaction.

orders['order_month'] = orders.apply(lambda order:order.order_date[:7], axis=1)
orders.columns

Index(['order_id', 'order_date', 'order_customer_id', 'order_status',
       'order_month'],
      dtype='object')

In [12]:
# column is created we can see the data as below 

orders

,order_id,order_date,order_customer_id,order_status,order_month
0,1,2013-07-25 00:00:00.0,11599,CLOSED,2013-07
1,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT,2013-07
2,3,2013-07-25 00:00:00.0,12111,COMPLETE,2013-07
3,4,2013-07-25 00:00:00.0,8827,CLOSED,2013-07
4,5,2013-07-25 00:00:00.0,11318,COMPLETE,2013-07
...,...,...,...,...,...
68878,68879,2014-07-09 00:00:00.0,778,COMPLETE,2014-07
68879,68880,2014-07-13 00:00:00.0,1117,COMPLETE,2014-07
68880,68881,2014-07-19 00:00:00.0,2518,PENDING_PAYMENT,2014-07
68881,68882,2014-07-22 00:00:00.0,10000,ON_HOLD,2014-07


In [13]:
# Now we can aggreate data based on new column.
groupby_column_list = ["order_month", "order_status"]

orders. \
        groupby(groupby_column_list)['order_id']. \
        agg(order_count = 'count')  # type: ignore

order_count
order_month order_status                
2013-07     CANCELED                  22
            CLOSED                   161
            COMPLETE                 515
            ON_HOLD                   81
            PAYMENT_REVIEW            19
...                                  ...
2014-07     PAYMENT_REVIEW            54
            PENDING                  517
            PENDING_PAYMENT          979
            PROCESSING               561
            SUSPECTED_FRAUD          101

[117 rows x 1 columns]

In [16]:
# To get system generated index in the result we wil use rest_index function

orders. \
        groupby(groupby_column_list)['order_id']. \
        agg(order_count = 'count').reset_index()      # type: ignore
          


,order_month,order_status,order_count
0,2013-07,CANCELED,22
1,2013-07,CLOSED,161
2,2013-07,COMPLETE,515
3,2013-07,ON_HOLD,81
4,2013-07,PAYMENT_REVIEW,19
...,...,...,...
112,2014-07,PAYMENT_REVIEW,54
113,2014-07,PENDING,517
114,2014-07,PENDING_PAYMENT,979
115,2014-07,PROCESSING,561
